# Finetune a YOLO model using a dataset from OpenStreetMap data and Mapbox tiles

## Dependencies

In [1]:
%pip install --quiet ultralytics

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 921.5/921.5 kB 19.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 70.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 59.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 39.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 11.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 29.1 MB/s eta 0:00:00


## Download dataset from HuggingFace Hub Datasets

You need to set the `HF_TOKEN` Colab secret:

- Create an account: https://huggingface.co/join
- Follow this guide about [`User Access Tokens`](https://huggingface.co/docs/hub/security-tokens)

In [2]:
from huggingface_hub import hf_hub_download

In [3]:
DATASET = "mozilla-ai/osm-swimming-pools"

In [4]:
hf_hub_download(
    DATASET, filename="train.zip", repo_type="dataset", local_dir="datasets"
)

train.zip:   0%|          | 0.00/419M [00:00<?, ?B/s]

'datasets/train.zip'

In [5]:
hf_hub_download(DATASET, filename="val.zip", repo_type="dataset", local_dir="datasets")

val.zip:   0%|          | 0.00/19.3M [00:00<?, ?B/s]

'datasets/val.zip'

In [6]:
hf_hub_download(
    DATASET, filename="yolo_dataset.yaml", repo_type="dataset", local_dir="datasets"
)

yolo_dataset.yaml:   0%|          | 0.00/73.0 [00:00<?, ?B/s]

'datasets/yolo_dataset.yaml'

In [7]:
!unzip -q datasets/train.zip

In [8]:
!unzip -q datasets/val.zip

# Finetune model

In [9]:
from ultralytics import YOLO

Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


Check the [available models](https://docs.ultralytics.com/tasks/detect/#models).

In [10]:
MODEL = "yolo11m.pt"

In [11]:
yolo = YOLO(MODEL)

100%|██████████| 38.8M/38.8M [00:00<00:00, 137MB/s]


In [ ]:
yolo.train(
    data="datasets/yolo_dataset.yaml",
    patience=3,
    imgsz=512,
    scale=0.0,
    flipud=0.5,
    translate=0.0,
)

# Check results

In [15]:
from PIL import Image

In [ ]:
Image.open("/content/runs/detect/train/val_batch0_pred.jpg")

# Upload model

In [18]:
USER = "mozilla-ai"
REPO = "swimming-pool-detector"

In [19]:
import yaml

with open("datasets/yolo_dataset.yaml") as f:
    CLASS_NAME = yaml.safe_load(f)["names"][0]

In [26]:
from pathlib import Path

Path("README.md").write_text(
    f"""
---
datasets:
- {DATASET}
base_model: Ultralytics/YOLO11
library_name: ultralytics
pipeline_tag: object-detection
license: apache-2.0
---

# {REPO}

Detect {CLASS_NAME}s in satellite images.

Created with [osm-ai-helper](https://github.com/mozilla-ai/osm-ai-helper).

`{MODEL}` trained with [ultralytics](https://docs.ultralytics.com/tasks/detect/).

![Predictions](./val_batch0_pred.jpg)
"""
)

430

In [21]:
from google.colab import userdata
from huggingface_hub import HfApi

In [22]:
api = HfApi()

In [23]:
api.create_repo(f"{USER}/{REPO}", token=userdata.get("HF_TOKEN"), repo_type="model")

RepoUrl('https://huggingface.co/mozilla-ai/swimming-pool-detector', endpoint='https://huggingface.co', repo_type='model', repo_id='mozilla-ai/swimming-pool-detector')

In [24]:
api.upload_file(
    token=userdata.get("HF_TOKEN"),
    path_or_fileobj="/content/runs/detect/train/weights/best.pt",
    path_in_repo="model.pt",
    repo_id=f"{USER}/{REPO}",
    repo_type="model",
)

best.pt:   0%|          | 0.00/40.5M [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/mozilla-ai/swimming-pool-detector/commit/419c9f7643c62026e8559425c3519726e56c892c', commit_message='Upload model.pt with huggingface_hub', commit_description='', oid='419c9f7643c62026e8559425c3519726e56c892c', pr_url=None, repo_url=RepoUrl('https://huggingface.co/mozilla-ai/swimming-pool-detector', endpoint='https://huggingface.co', repo_type='model', repo_id='mozilla-ai/swimming-pool-detector'), pr_revision=None, pr_num=None)

In [28]:
api.upload_file(
    token=userdata.get("HF_TOKEN"),
    path_or_fileobj="README.md",
    path_in_repo="README.md",
    repo_id=f"{USER}/{REPO}",
    repo_type="model",
)

No files have been modified since last commit. Skipping to prevent empty commit.


CommitInfo(commit_url='https://huggingface.co/mozilla-ai/swimming-pool-detector/commit/6e74b0cfd854dc8edc8f814fdcab8079691a5291', commit_message='Upload README.md with huggingface_hub', commit_description='', oid='6e74b0cfd854dc8edc8f814fdcab8079691a5291', pr_url=None, repo_url=RepoUrl('https://huggingface.co/mozilla-ai/swimming-pool-detector', endpoint='https://huggingface.co', repo_type='model', repo_id='mozilla-ai/swimming-pool-detector'), pr_revision=None, pr_num=None)

In [29]:
api.upload_file(
    token=userdata.get("HF_TOKEN"),
    path_or_fileobj="/content/runs/detect/train/val_batch0_pred.jpg",
    path_in_repo="val_batch0_pred.jpg",
    repo_id=f"{USER}/{REPO}",
    repo_type="model",
)

val_batch0_pred.jpg:   0%|          | 0.00/695k [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/mozilla-ai/swimming-pool-detector/commit/998e0f37bad7d5a4d7ccb7656e33585278bc8ef4', commit_message='Upload val_batch0_pred.jpg with huggingface_hub', commit_description='', oid='998e0f37bad7d5a4d7ccb7656e33585278bc8ef4', pr_url=None, repo_url=RepoUrl('https://huggingface.co/mozilla-ai/swimming-pool-detector', endpoint='https://huggingface.co', repo_type='model', repo_id='mozilla-ai/swimming-pool-detector'), pr_revision=None, pr_num=None)